In [1]:
import os

GLOBUS_CLIENT_ID = None
GLOBUS_CLIENT_SECRET = None

CHI_PROJ_NAME = "CHI-231082"
LEASE_DURATION = 2  # units?
N_DEVICES = 10

DOCKER_IMAGE = "valhayot/flox-images:latest"

CONTAINER_ENV = {
    "foo": "bar",
    # "FUNCX_SDK_CLIENT_ID": GLOBUS_CLIENT_ID,
    # "FUNCX_SDK_CLIENT_SECRET": GLOBUS_CLIENT_SECRET,
    # "PROXYSTORE_GLOBUS_CLIENT_ID": GLOBUS_CLIENT_ID,
    # "PROXYSTORE_GLOBUS_CLIENT_SECRET": GLOBUS_CLIENT_SECRET,
}

# CONTAINER_PORTS = [5671, 8675, 6734, 5263, 6736, 7563, 6476, 7864, 8943, 8426]
CONTAINER_PORTS = [5671, 8675]

USERNAME = "hudsonn"
MACHINE_NAME = "raspberrypi4-64"
RESERVATION = "3997075c-0a2c-4343-ae7c-f13486226ad9"

In [2]:
import chi
import chi_edge

print(chi_edge.SUPPORTED_MACHINE_NAMES)

chi.set("project_domain_name", "chameleon")
chi.set("project_name", CHI_PROJ_NAME)
chi.use_site("CHI@Edge")

{'raspberrypi3-64': 'Raspberry Pi 3 (using 64bit OS)', 'raspberrypi4-64': 'Raspberry Pi 4 (using 64bit OS)', 'jetson-nano': 'Jetson Nano Devkit (SD Card, rev B01)', 'jetson-xavier-nx-emmc': 'Jetson Xavier NX Devkit (eMMC)'}
Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org


In [3]:
from chi import container, lease


def create_reservation(
        username: str,
        machine_name: str,
        lease_days: int = 1,
        lease_hours: int = 0,
        n_devices: int = 1,
):
    start, end = lease.lease_duration(days=lease_days, hours=lease_hours)
    lease_name = f"{username}-{machine_name}-{start}"

    reservations = []
    lease.add_device_reservation(reservations, count=n_devices, device_name=machine_name)
    container_lease = lease.create_lease(lease_name, reservations)
    lease_id = container_lease["id"]
    reservation_id = lease.get_device_reservation(lease_id)

    print(
        f"❯ Created lease with {lease_name=}, uuid={lease_id}, and {reservation_id=}... "
        f"Waiting for it to start. This can take up to 60 seconds."
    )

    lease.wait_for_active(lease_id)
    print("❯ Done!")

    return reservation_id


def create_container(
        username: str,
        machine_name: str,
        reservation_id: str,
        docker_image,
        container_ports,
        container_env,
        name_suffix: str | None = None,
):
    print("❯ Requesting container...")
    if name_suffix is not None:
        container_name = f"{username}-{machine_name}-{name_suffix}".replace("_", "-")
    else:
        container_name = f"{username}-{machine_name}".replace("_", "-")

    try:
        my_container = container.create_container(
            container_name,
            image=docker_image,
            exposed_ports=container_ports,
            environment=container_env,
            reservation_id=reservation_id,
            platform_version=2,
        )
    except RuntimeError as err:
        print(err)
        print(f"❯ [ERR] Please stop and/or delete {container_name} and try again.")
    else:
        print(f"❯ Successfully created container: {container_name}!")

    return my_container


In [7]:
# if RESERVATION is None:
#     RESERVATION = create_reservation(USERNAME, MACHINE_NAME, n_devices=N_DEVICES)

['allow', 'deny']

In [4]:
workers = []
for i in range(N_DEVICES):
    w = create_container(
        USERNAME, MACHINE_NAME, RESERVATION, DOCKER_IMAGE,
        CONTAINER_PORTS, CONTAINER_ENV, name_suffix=f"{i}"
    )
    workers.append(w)

workers

❯ Requesting container...


BadRequest: Invalid input for field 'identity/token/id': None is not of type 'string'

Failed validating 'type' in schema['properties']['identity']['properties']['token']['properties']['id']:
    {'type': 'string'}

On instance['identity']['token']['id']:
    None (HTTP 400) (Request-ID: req-397902ac-ba14-467b-b772-87083f491b5d)